# Cross-Language Code Converter & Performance Benchmark

This notebook takes a **JavaScript** algorithm, uses an LLM to convert it to **C# (.NET)**, **Python**, and **Java**, then runs all four versions and compares execution speed — all through a **Gradio web UI**.

### How it works
1. Paste or edit a JavaScript snippet in the UI
2. Click **Convert & Benchmark** — the LLM converts it to each target language
3. Each version is executed with its native runtime and timed
4. Results are displayed as a ranked table, bar chart, and code tabs

### Prerequisites

**Required:**
```
pip install gradio openai python-dotenv matplotlib
```

**Runtimes (install the ones you want to benchmark):**

| Language | Runtime | Check |
|----------|---------|-------|
| JavaScript | `node` | `node --version` |
| Python | `python3` | `python3 --version` |
| C# (.NET) | `dotnet` (SDK) | `dotnet --version` |
| Java | `javac` + `java` (JDK) | `java --version` |

> Missing runtimes are skipped gracefully, the app still runs with whatever is available.

### Environment
```
OPENROUTER_API_KEY=sk-or-v1-...
```

In [ ]:
import os
import re
import shutil
import subprocess
import tempfile
import time
from dataclasses import dataclass
from typing import Optional

import gradio as gr
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)
MODEL = "openai/gpt-4o-mini"

LANGUAGES = {
    "javascript": "JavaScript",
    "python": "Python",
    "csharp": "C#",
    "java": "Java",
}

RUNTIMES: dict[str, bool] = {
    "javascript": shutil.which("node") is not None,
    "python": shutil.which("python3") is not None,
    "csharp": shutil.which("dotnet") is not None,
    "java": shutil.which("javac") is not None and shutil.which("java") is not None,
}

print("Available runtimes:")
for lang, available in RUNTIMES.items():
    status = "ready" if available else "NOT FOUND — will be skipped"
    print(f"  {lang:12s} {status}")

In [ ]:
# The JavaScript source code to convert and benchmark.
# Two CPU-bound algorithms: prime sieve and matrix multiplication.

JS_SOURCE = r"""
function sieveOfEratosthenes(limit) {
    const isPrime = new Array(limit + 1).fill(true);
    isPrime[0] = isPrime[1] = false;
    for (let i = 2; i * i <= limit; i++) {
        if (isPrime[i]) {
            for (let j = i * i; j <= limit; j += i) {
                isPrime[j] = false;
            }
        }
    }
    let count = 0;
    for (let i = 0; i <= limit; i++) {
        if (isPrime[i]) count++;
    }
    return count;
}

function matrixMultiply(size) {
    const A = [], B = [], C = [];
    for (let i = 0; i < size; i++) {
        A[i] = []; B[i] = []; C[i] = [];
        for (let j = 0; j < size; j++) {
            A[i][j] = Math.random();
            B[i][j] = Math.random();
            C[i][j] = 0;
        }
    }
    for (let i = 0; i < size; i++)
        for (let j = 0; j < size; j++)
            for (let k = 0; k < size; k++)
                C[i][j] += A[i][k] * B[k][j];
    return C[0][0];
}

const SIEVE_LIMIT = 5_000_000;
const MATRIX_SIZE = 200;

const t1 = Date.now();
const primeCount = sieveOfEratosthenes(SIEVE_LIMIT);
const sieveMs = Date.now() - t1;

const t2 = Date.now();
const matResult = matrixMultiply(MATRIX_SIZE);
const matMs = Date.now() - t2;

console.log(`Sieve: ${primeCount} primes in ${sieveMs}ms`);
console.log(`Matrix: result=${matResult.toFixed(4)} in ${matMs}ms`);
console.log(`TOTAL_MS:${sieveMs + matMs}`);
""".strip()

print(JS_SOURCE)

In [ ]:
# LLM-powered code conversion

CONVERSION_SYSTEM_PROMPT = (
    "You are an expert polyglot programmer. Convert the given JavaScript code "
    "to the requested language. Rules:\n"
    "1. Preserve EXACT algorithm logic — same sieve limit, same matrix size.\n"
    "2. The converted code must be a complete, runnable file (no external deps).\n"
    "3. It must print timing in the EXACT format: TOTAL_MS:<number>\n"
    "4. Use the language's standard timing mechanism (not external libraries).\n"
    "5. Output ONLY the code inside a single code block — no explanation.\n"
    "6. For C#: use a top-level program (no class/namespace wrapper). Use System.Diagnostics.Stopwatch.\n"
    "7. For Java: use a single public class named Main. Use System.nanoTime().\n"
    "8. For Python: use time.perf_counter(). Use standard lists (no numpy).\n"
    "9. Use random numbers for matrix init (same as JS Math.random())."
)


def convert_code(source_js: str, target_language: str) -> str:
    """Ask the LLM to convert JS code to the target language. Returns raw code string."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": CONVERSION_SYSTEM_PROMPT},
            {"role": "user", "content": (
                f"Convert this JavaScript to **{target_language}**:\n\n"
                f"```javascript\n{source_js}\n```"
            )},
        ],
        temperature=0,
    )
    raw = response.choices[0].message.content

    # Extract code from markdown fences if present
    match = re.search(r"```(?:\w+)?\n(.*?)```", raw, re.DOTALL)
    return match.group(1).strip() if match else raw.strip()


# Test with a quick conversion preview
preview = convert_code(JS_SOURCE, "Python")
print("=== Python preview (first 20 lines) ===")
print("\n".join(preview.splitlines()[:20]))

In [ ]:
# Code execution engine — runs each language and captures timing

@dataclass
class BenchmarkResult:
    language: str
    code: str
    stdout: str = ""
    stderr: str = ""
    exit_code: int = -1
    wall_time_ms: float = 0.0
    internal_time_ms: float = 0.0
    error: Optional[str] = None


def _extract_internal_time(stdout: str) -> float:
    """Parse TOTAL_MS:<number> from program output."""
    match = re.search(r"TOTAL_MS[:\s]*([\d.]+)", stdout)
    return float(match.group(1)) if match else 0.0


def run_javascript(code: str) -> BenchmarkResult:
    result = BenchmarkResult(language="JavaScript", code=code)
    with tempfile.NamedTemporaryFile(suffix=".js", mode="w", delete=False) as f:
        f.write(code)
        f.flush()
        start = time.perf_counter()
        proc = subprocess.run(["node", f.name], capture_output=True, text=True, timeout=120)
        result.wall_time_ms = (time.perf_counter() - start) * 1000
    result.stdout, result.stderr, result.exit_code = proc.stdout, proc.stderr, proc.returncode
    result.internal_time_ms = _extract_internal_time(proc.stdout)
    os.unlink(f.name)
    return result


def run_python(code: str) -> BenchmarkResult:
    result = BenchmarkResult(language="Python", code=code)
    with tempfile.NamedTemporaryFile(suffix=".py", mode="w", delete=False) as f:
        f.write(code)
        f.flush()
        start = time.perf_counter()
        proc = subprocess.run(["python3", f.name], capture_output=True, text=True, timeout=300)
        result.wall_time_ms = (time.perf_counter() - start) * 1000
    result.stdout, result.stderr, result.exit_code = proc.stdout, proc.stderr, proc.returncode
    result.internal_time_ms = _extract_internal_time(proc.stdout)
    os.unlink(f.name)
    return result


def run_csharp(code: str) -> BenchmarkResult:
    result = BenchmarkResult(language="C# (.NET)", code=code)
    tmpdir = tempfile.mkdtemp(prefix="cs_bench_")
    try:
        # Create a minimal console project
        subprocess.run(
            ["dotnet", "new", "console", "-o", tmpdir, "--force"],
            capture_output=True, timeout=30,
        )
        # Overwrite Program.cs with our code
        with open(os.path.join(tmpdir, "Program.cs"), "w") as f:
            f.write(code)

        start = time.perf_counter()
        proc = subprocess.run(
            ["dotnet", "run", "--project", tmpdir, "-c", "Release"],
            capture_output=True, text=True, timeout=120,
        )
        result.wall_time_ms = (time.perf_counter() - start) * 1000
        result.stdout, result.stderr, result.exit_code = proc.stdout, proc.stderr, proc.returncode
        result.internal_time_ms = _extract_internal_time(proc.stdout)
    finally:
        shutil.rmtree(tmpdir, ignore_errors=True)
    return result


def run_java(code: str) -> BenchmarkResult:
    result = BenchmarkResult(language="Java", code=code)
    tmpdir = tempfile.mkdtemp(prefix="java_bench_")
    try:
        filepath = os.path.join(tmpdir, "Main.java")
        with open(filepath, "w") as f:
            f.write(code)

        # Compile
        comp = subprocess.run(
            ["javac", filepath], capture_output=True, text=True, timeout=30,
        )
        if comp.returncode != 0:
            result.stderr = comp.stderr
            result.exit_code = comp.returncode
            result.error = "Compilation failed"
            return result

        # Run
        start = time.perf_counter()
        proc = subprocess.run(
            ["java", "-cp", tmpdir, "Main"],
            capture_output=True, text=True, timeout=120,
        )
        result.wall_time_ms = (time.perf_counter() - start) * 1000
        result.stdout, result.stderr, result.exit_code = proc.stdout, proc.stderr, proc.returncode
        result.internal_time_ms = _extract_internal_time(proc.stdout)
    finally:
        shutil.rmtree(tmpdir, ignore_errors=True)
    return result


RUNNERS = {
    "javascript": run_javascript,
    "python": run_python,
    "csharp": run_csharp,
    "java": run_java,
}

print("Execution engine ready.")

In [ ]:
# Gradio pipeline — convert, benchmark, visualize

def _build_chart(results: list[BenchmarkResult]):
    """Create a matplotlib bar chart from benchmark results."""
    successful = [r for r in results if r.exit_code == 0 and r.internal_time_ms > 0]
    if not successful:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.text(0.5, 0.5, "No successful runs to chart", ha="center", va="center", fontsize=14)
        ax.set_axis_off()
        return fig

    successful.sort(key=lambda r: r.internal_time_ms)
    langs = [r.language for r in successful]
    times = [r.internal_time_ms for r in successful]
    colors = ["#2ecc71" if t == min(times) else "#3498db" for t in times]

    fig, ax = plt.subplots(figsize=(8, 4))
    bars = ax.barh(langs, times, color=colors, edgecolor="white", height=0.5)
    ax.set_xlabel("Execution Time (ms)")
    ax.set_title("Language Performance Comparison (lower is better)")
    ax.invert_yaxis()

    for bar, t in zip(bars, times):
        ax.text(bar.get_width() + max(times) * 0.02, bar.get_y() + bar.get_height() / 2,
                f"{t:.0f}ms", va="center", fontweight="bold")
    plt.tight_layout()
    return fig


def run_pipeline(js_code: str, num_runs: int, progress=gr.Progress()):
    """Full pipeline: convert JS to all languages, benchmark, return all UI outputs."""
    num_runs = int(num_runs)
    log_lines: list[str] = []

    # --- Phase 1: Convert ---
    converted: dict[str, str] = {"javascript": js_code.strip()}

    for lang_key, lang_name in LANGUAGES.items():
        if lang_key == "javascript":
            continue
        if not RUNTIMES[lang_key]:
            log_lines.append(f"[SKIP] {lang_name} — runtime not installed")
            continue

        progress(0, desc=f"Converting JS to {lang_name}...")
        log_lines.append(f"[CONVERT] JS to {lang_name}...")
        try:
            code = convert_code(js_code, lang_name)
            converted[lang_key] = code
            log_lines.append(f"   [OK] {lang_name} — {len(code)} chars")
        except Exception as e:
            log_lines.append(f"   [FAIL] {lang_name} conversion failed: {e}")

    # --- Phase 2: Benchmark ---
    results: list[BenchmarkResult] = []
    total_runs = len(converted) * num_runs
    completed = 0

    for lang_key, code in converted.items():
        lang_name = LANGUAGES[lang_key]
        runner = RUNNERS[lang_key]
        best: Optional[BenchmarkResult] = None

        for i in range(num_runs):
            progress(completed / total_runs, desc=f"Running {lang_name} ({i + 1}/{num_runs})...")
            try:
                r = runner(code)
                run_status = f"exit={r.exit_code}, wall={r.wall_time_ms:.0f}ms, internal={r.internal_time_ms:.0f}ms"
                log_lines.append(f"[RUN] {lang_name} run {i + 1}: {run_status}")

                if r.exit_code != 0:
                    log_lines.append(f"   [WARN] {r.stderr[:200]}")
                    if best is None:
                        best = r
                elif best is None or r.internal_time_ms < best.internal_time_ms:
                    best = r

            except subprocess.TimeoutExpired:
                log_lines.append(f"[TIMEOUT] {lang_name} run {i + 1}: timed out")
            except Exception as e:
                log_lines.append(f"[ERROR] {lang_name} run {i + 1}: {e}")
            completed += 1

        if best:
            results.append(best)

    progress(1.0, desc="Done!")

    # --- Build outputs ---
    results.sort(key=lambda r: r.internal_time_ms if r.exit_code == 0 else float("inf"))

    # Summary line
    if len(results) >= 2:
        fastest, slowest = results[0], results[-1]
        if fastest.internal_time_ms > 0:
            speedup = slowest.internal_time_ms / fastest.internal_time_ms
            log_lines.append(f"\n>> WINNER: {fastest.language} is {speedup:.1f}x faster than {slowest.language}")

    # Table data for gr.Dataframe
    table_rows = []
    for i, r in enumerate(results, 1):
        table_rows.append([
            i,
            r.language,
            round(r.internal_time_ms),
            round(r.wall_time_ms),
            "OK" if r.exit_code == 0 else "FAIL",
            r.stdout.strip().replace("\n", " | ")[:100],
        ])

    chart = _build_chart(results)
    status_log = "\n".join(log_lines)

    return (
        converted.get("javascript", ""),
        converted.get("python", "— not converted —"),
        converted.get("csharp", "— not converted —"),
        converted.get("java", "— not converted —"),
        table_rows,
        chart,
        status_log,
    )

In [ ]:
# Gradio UI — launch the app

runtime_status = " | ".join(
    f"**{LANGUAGES[k]}**: {'yes' if v else 'no'}" for k, v in RUNTIMES.items()
)

with gr.Blocks(title="Code Converter & Benchmark", theme=gr.themes.Soft()) as app:
    gr.Markdown(
        "# Cross-Language Code Converter & Benchmark\n"
        "Paste JavaScript code, convert it to Python / C# / Java via LLM, "
        "then benchmark all versions head-to-head.\n\n"
        f"**Detected runtimes:** {runtime_status}"
    )

    with gr.Row():
        with gr.Column(scale=3):
            js_input = gr.Code(
                value=JS_SOURCE,
                language="javascript",
                label="JavaScript Source Code",
                lines=20,
            )
        with gr.Column(scale=1):
            num_runs = gr.Slider(1, 10, value=3, step=1, label="Runs per language")
            run_btn = gr.Button("Convert & Benchmark", variant="primary", size="lg")

    gr.Markdown("---")
    gr.Markdown("### Results")

    with gr.Row():
        with gr.Column(scale=3):
            results_table = gr.Dataframe(
                headers=["Rank", "Language", "Internal (ms)", "Wall (ms)", "Status", "Output"],
                label="Benchmark Results",
                interactive=False,
            )
        with gr.Column(scale=2):
            chart_output = gr.Plot(label="Performance Chart")

    gr.Markdown("### Converted Code")

    with gr.Tabs():
        with gr.Tab("JavaScript"):
            code_js = gr.Code(language="javascript", label="JavaScript", interactive=False)
        with gr.Tab("Python"):
            code_py = gr.Code(language="python", label="Python", interactive=False)
        with gr.Tab("C#"):
            code_cs = gr.Code(language="c", label="C#", interactive=False)
        with gr.Tab("Java"):
            code_java = gr.Code(language="c", label="Java", interactive=False)

    with gr.Accordion("Execution Log", open=False):
        log_output = gr.Textbox(label="Log", lines=15, interactive=False)

    run_btn.click(
        fn=run_pipeline,
        inputs=[js_input, num_runs],
        outputs=[code_js, code_py, code_cs, code_java, results_table, chart_output, log_output],
    )

app.launch(inbrowser=True)